In [1]:
from dask.distributed import Client, progress
# HPC
# client = Client(scheduler_file='/glade/scratch/jhamman/scheduler.json')
# client

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

In [2]:
client = Client(cluster)
client

Client Scheduler: tcp://10.32.60.90:43455 Dashboard: /user/0000-0002-9124-2827/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.crs as ccrs

import sys
### general purpose
import os
import xarray as xr
import pandas as pd
### palette
import matplotlib.cm as mplcm
#import matplotlib.cm as cm
#div_cmap = mplcm.seismic
import cmocean
from cmocean import cm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import scipy.io as sio

import time

In [4]:
import gcsfs

fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')
gcs = gcsfs.GCSFileSystem(gcs=fs,check=False,create=False)
gcsmapgrid=gcs.get_mapper('pangeo-data/eNATL60-I/zarr_coord_eNATL60')
dsgrid_eNATL=xr.open_zarr(gcsmapgrid)

In [5]:
lat=dsgrid_eNATL.nav_lat
lon=dsgrid_eNATL.nav_lon

In [6]:
gcsmapgrid_ORCA = gcs.get_mapper('pangeo-data/eORCA1-I/zarr_IPSLCM6ALR_eORCA1.2_mesh_mask')
dsgrid_ORCA = xr.open_zarr(gcsmapgrid_ORCA)

In [7]:
lat_ORCA=dsgrid_ORCA.nav_lat
lon_ORCA=dsgrid_ORCA.nav_lon

In [8]:
latmin = 56.; latmax = 61.
lonmin = -55.; lonmax = -50.

In [9]:
def read_datagrid(lat,lon,latmin=None,latmax=None,lonmin=None,lonmax=None):
    domain = (lonmin<lon.values) * (lon.values<lonmax) * (latmin<lat.values) * (lat.values<latmax)
    where = np.where(domain)
    vlats = lat.values[where]
    vlons = lon.values[where]
    #get indice
    jmin = where[0][vlats.argmin()]
    jmax = where[0][vlats.argmax()]
    imin = where[1][vlons.argmin()]
    imax = where[1][vlons.argmax()]
    navlon = lon.values[jmin:jmax+1,imin:imax+1]
    navlat = lat.values[jmin:jmax+1,imin:imax+1]
    return navlon,navlat,jmin,jmax,imin,imax

In [10]:
domain = (lonmin<lon_ORCA) * (lon_ORCA<lonmax) * (latmin<lat_ORCA) * (lat_ORCA<latmax)
where = np.where(domain)
vlats = lat_ORCA.values[where]
vlons = lon_ORCA.values[where]
#get indice
jmin_ORCA = where[0][vlats.argmin()]
jmax_ORCA = where[0][vlats.argmax()]
imin_ORCA = where[1][vlons.argmin()]
imax_ORCA = where[1][vlons.argmax()]
#load arrays
navlon_ORCA = lon_ORCA.values[jmin_ORCA:jmax_ORCA+1,imin_ORCA:imax_ORCA+1]
navlat_ORCA = lat_ORCA.values[jmin_ORCA:jmax_ORCA+1,imin_ORCA:imax_ORCA+1]

In [ ]:
# - Define box dimensions
a = (np.shape(navlon_ORCA)[0]-1.)*(np.shape(navlat_ORCA)[1]-1.)
Box_arr = np.empty((int(a),6),dtype=np.dtype('U100'))
print(a)

ii = 0
jj = 0
for i in np.arange(0,int(a),1):
    Box_arr[i,:] = [str(navlat_ORCA[ii,jj]),str(navlat_ORCA[ii+1,jj]),str(navlon_ORCA[ii,jj]),str(navlon_ORCA[ii,jj+1]),'Box_'+str(i),str(i)]
    ii = ii + 1
    if ii >= int(np.shape(navlon_ORCA)[0])-1:
        ii = 0
        jj = jj + 1

name = Box_arr[:,4]

#- defining dictionaries for the boxes
class box: # empty container.
    def __init__(self,name=None):
        self.name = name
        return

dictboxes = {}

for i in np.arange(0,int(a),1):
    
    y1 = float(Box_arr[i,0]); y2 = float(Box_arr[i,1]);
    x2 = float(Box_arr[i,2]); x1 = float(Box_arr[i,3]);
    box_name = Box_arr[i,4]
    print(box_name)
    
    navlon,navlat,jmin,jmax,imin,imax = read_datagrid(lat,lon,latmin=y1,latmax=y2,lonmin=x2,lonmax=x1)
    
    # - save box parameter
    abox = box(box_name)
    abox.lonmin = navlon.min()
    abox.lonmax = navlon.max()
    abox.latmin = navlat.min()
    abox.latmax = navlat.max()
    abox.navlon = navlon
    abox.navlat = navlat
    abox.imin = imin
    abox.imax = imax
    abox.jmin = jmin
    abox.jmax = jmax
    abox.nb = int(Box_arr[i,5])#ibox[5]
    dictboxes[box_name] = abox
    
boxes = dictboxes.values()

for box in boxes:
    print(box.name,box.imin,box.imax,box.jmin,box.jmax)

40.0
Box_0
Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
Box_15
Box_16
Box_17
Box_18
Box_19
Box_20
Box_21
Box_22
Box_23
Box_24
Box_25
Box_26
Box_27
Box_28
Box_29
Box_30


In [ ]:
transform = ccrs.PlateCarree()

pltargs = {
   'transform' : transform,
   'color' : 'black',
   'linewidth' : 2}

def plot_individual_box(box,ax=None):
    x1 = box.lonmin
    x2 = box.lonmax
    y1 = box.latmin
    y2 = box.latmax
    ax.plot([x1, x2], [y1, y1],**pltargs)
    ax.plot([x1, x2], [y2, y2],**pltargs)
    ax.plot([x1, x1], [y1, y2],**pltargs)
    ax.plot([x2, x2], [y1, y2],**pltargs)
    #ax.text(x1+0.1,y1+0.1,box.nb)


def plot_boxes(boxlist,textbox=None,title=None):
    plt.figure(figsize=(15,10))
    proj = transform
    ax = plt.axes(projection=proj)
    ax.stock_img()
    for box in boxlist:
        plot_individual_box(box,ax=ax)
    ax.coastlines(resolution="10m")

    # grid lines
    gl = ax.gridlines(draw_labels=True,
                  linewidth=1, color='black', alpha=0.2, linestyle='--')

    label_style = {'size': 12, 'color': 'black', 'weight': 'bold'}

    gl.xlabel_style = label_style
    gl.xlabels_bottom = False
    gl.xlocator = mticker.FixedLocator(np.arange(-90,10,5,dtype=float))

    gl.ylabel_style = label_style
    gl.ylabels_right = False
    gl.ylocator = mticker.FixedLocator(np.arange(20,80,5,dtype=float))


    #ax.grid(False)
    plt.xlim((-85,5))
    plt.ylim((25,70))
    if textbox is not None:
        # see http://matplotlib.org/users/annotations_guide.html
        props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor="black", lw=2)
        ax.text(0.05, 0.10, textbox, transform=ax.transAxes, fontsize=18,verticalalignment='top',bbox=props)
    if title is not None:
        plt.title(title,fontsize=20,y=1.08)

       
plot_boxes(boxes,textbox=None,title=None)
plt.savefig('test_map_boxes_1x1_LS.png')